In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([59, 65, 43, 14, 82, 63, 37, 33, 53, 57, 57, 57, 23, 43, 14, 14, 17,
       33, 64, 43, 46, 80, 50, 80, 63, 49, 33, 43, 37, 63, 33, 43, 50, 50,
       33, 43, 50, 80, 24, 63, 58, 33, 63, 56, 63, 37, 17, 33, 39, 34, 65,
       43, 14, 14, 17, 33, 64, 43, 46, 80, 50, 17, 33, 80, 49, 33, 39, 34,
       65, 43, 14, 14, 17, 33, 80, 34, 33, 80, 82, 49, 33, 11, 21, 34, 57,
       21, 43, 17,  4, 57, 57, 61, 56, 63, 37, 17, 82, 65, 80, 34])

In [35]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    # Fill the appropriate elements with ones
    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    print(one_hot)
    print("")
    return one_hot

In [36]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

#print(one_hot)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]



In [17]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [18]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [19]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[36 23 43 35 63 50 17  3 77 32]
 [28 15  2  3 63 23 43 63  3 43]
 [50  2 11  3 15 17  3 43  3 62]
 [28  3 63 23 50  3 29 23 69 50]
 [ 3 28 43  6  3 23 50 17  3 63]
 [29  1 28 28 69 15  2  3 43  2]
 [ 3 46  2  2 43  3 23 43 11  3]
 [10 57 16 15  2 28 66 18 58  3]]

y
 [[23 43 35 63 50 17  3 77 32 32]
 [15  2  3 63 23 43 63  3 43 63]
 [ 2 11  3 15 17  3 43  3 62 15]
 [ 3 63 23 50  3 29 23 69 50 62]
 [28 43  6  3 23 50 17  3 63 50]
 [ 1 28 28 69 15  2  3 43  2 11]
 [46  2  2 43  3 23 43 11  3 28]
 [57 16 15  2 28 66 18 58  3 75]]


In [20]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [21]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [22]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [23]:
#define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [24]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2825... Val Loss: 3.2288
Epoch: 1/20... Step: 20... Loss: 3.1557... Val Loss: 3.1404
Epoch: 1/20... Step: 30... Loss: 3.1452... Val Loss: 3.1261
Epoch: 1/20... Step: 40... Loss: 3.1145... Val Loss: 3.1197
Epoch: 1/20... Step: 50... Loss: 3.1462... Val Loss: 3.1180
Epoch: 1/20... Step: 60... Loss: 3.1209... Val Loss: 3.1163
Epoch: 1/20... Step: 70... Loss: 3.1102... Val Loss: 3.1151
Epoch: 1/20... Step: 80... Loss: 3.1246... Val Loss: 3.1120
Epoch: 1/20... Step: 90... Loss: 3.1220... Val Loss: 3.1052
Epoch: 1/20... Step: 100... Loss: 3.0987... Val Loss: 3.0880
Epoch: 1/20... Step: 110... Loss: 3.0674... Val Loss: 3.0485
Epoch: 1/20... Step: 120... Loss: 2.9788... Val Loss: 2.9743
Epoch: 1/20... Step: 130... Loss: 2.8821... Val Loss: 2.9757
Epoch: 2/20... Step: 140... Loss: 2.8556... Val Loss: 2.7967
Epoch: 2/20... Step: 150... Loss: 2.7248... Val Loss: 2.6726
Epoch: 2/20... Step: 160... Loss: 2.6352... Val Loss: 2.5918
Epoch: 2/20... Step: 170... Loss:

Epoch: 19/20... Step: 2640... Loss: 1.2296... Val Loss: 1.2872
Epoch: 20/20... Step: 2650... Loss: 1.2399... Val Loss: 1.2859
Epoch: 20/20... Step: 2660... Loss: 1.2362... Val Loss: 1.2896
Epoch: 20/20... Step: 2670... Loss: 1.2350... Val Loss: 1.2823
Epoch: 20/20... Step: 2680... Loss: 1.2273... Val Loss: 1.2789
Epoch: 20/20... Step: 2690... Loss: 1.2178... Val Loss: 1.2845
Epoch: 20/20... Step: 2700... Loss: 1.2294... Val Loss: 1.2752
Epoch: 20/20... Step: 2710... Loss: 1.1949... Val Loss: 1.2764
Epoch: 20/20... Step: 2720... Loss: 1.2023... Val Loss: 1.2770
Epoch: 20/20... Step: 2730... Loss: 1.1892... Val Loss: 1.2765
Epoch: 20/20... Step: 2740... Loss: 1.1943... Val Loss: 1.2790
Epoch: 20/20... Step: 2750... Loss: 1.1925... Val Loss: 1.2800
Epoch: 20/20... Step: 2760... Loss: 1.1912... Val Loss: 1.2832
Epoch: 20/20... Step: 2770... Loss: 1.2328... Val Loss: 1.2745
Epoch: 20/20... Step: 2780... Loss: 1.2499... Val Loss: 1.2790


In [25]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [26]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [27]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [29]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna Alexey Alexandrovitth.

"You want to be all the most any care of the countries."

"Yes, I can't think of her, then you met her mother that's his for thriegs to the
rand, send me anyone. What are you with tome to those importance in his
mother, well it's a string then, who can't such been if these any
minuces and my country subject?" he thought, satting its fresh was the
sacondation as he had been to bring him to the
consciousness of the comiss, the poor
weather, and he felt in tears of her and the peasants, he heard the
panding shid in the fronce who was, and showen the place. The
soce table was a special pease of an excepress of his fresh and
secrets of the sounds and such a possibility of country, had to
be serroud at once at all over the chair, had sold her face. But this he
had to be destined.

And she was started as that has always been always that which had teen her
head and have nothing but she had not spent it would help, while
a suffering, and he went on. "I don't underst

In [30]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [31]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said,
stopped her, and went up and deconoundly as he could be a candle tatted
in the conversation.
"This should be the minutess!"

"Oh, I was anyway, it's away in the first come," said Levin.

"I should be always the motions with him only."

His brother would have the pail of an early to her true that
he was too the same some, and that he would not have here the baly
and so that the position had been asking him in a most child, to
the seaments of the princess. There had seen her the care of
the called an action what he went up, and another carriage,
saying what was the fiest of the part to the conversation, and
they were through his hands, but the personares, a man that their
certain the hand of a man who had seen her. The conversation was
consciaution, and who had the close of answering his heart. The
princess seemed to her and so much by the sourd of her son he
had been so told his back.

The same talk transloved his hand. To the semonious strone too, to
severathy impossibl